<a href="https://colab.research.google.com/github/d2525456d/Doantotnghiep/blob/main/Thu%E1%BA%ADt%20to%C3%A1n%20m%C3%B4%20ph%E1%BB%8Fng%20luy%E1%BB%87n%20kim/toiuu_canbang%E2%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import pandas as pd

# Load the uploaded file
file_path = '/content/data.csv'
df = pd.read_csv(file_path)

# Prepare data
tasks_times = df['TGLT'].tolist()
machines = [
    0, 0, 0, 0, 1, 2, 1, 1, 0, 2, 7, 1, 7, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 4, 1,
    5, 1, 1, 1, 1, 2, 7, 1, 1, 1, 1, 1, 2, 1, 1, 3, 2, 2, 7, 1, 1, 3, 7,
    6, 2, 0, 0, 0, 2, 1, 2, 4, 2, 4, 1, 1, 0, 1, 1, 1, 4, 0, 1, 1, 2, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0
]
df["Thiết bị"] = machines
proficiency_levels = {
    "Lê Thị Phương Thảo": 1.48, "Hoàng Thị Mỹ Yến": 1.16, "Trần Thị Tuyền": 1.48,
    "Ngô Khắc Hiển": 1.00, "Hồ Thị Diệp": 1.16, "Nguyễn Thị Mỹ Nhung": 1.16,
    "Thái Thị Nga": 1.16, "Trần Thị Cúc": 1.00, "Lê Thị Thanh Hà": 1.16,
    "Nguyễn Thị Thúy": 1.16, "Trần Thị Cẩm Rin": 1.32, "Hoàng Thị Nở": 1.16,
    "Lê Thị Như Quỳnh": 1.16, "Trần Thị Thương": 1.16, "Đinh Thị Mi": 1.16,
    "Nguyễn Thị Ngọc Lan": 1.16, "Bùi Thị Hồng Vân": 1.00, "Trần Thị Oanh": 1.16,
    "Nguyễn Thị Gấm": 1.32, "Phan Thị Thu Hoài": 1.16, "Nguyễn Thị Thu": 1.16,
    "Nguyễn Thị Hài": 1.16, "Nguyễn Thị Hồng": 1.48
}
workers = list(proficiency_levels.keys())

def calculate_fitness_with_product(individual, tasks_times, proficiency_levels, df):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = tasks_times[task_idx]
        n_workers = len(worker_list)

        theoretical_time_per_worker = task_time / n_workers
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency
            detailed_task_allocation[worker].append(worker_time)

    worker_time_summary = [
        sum(worker_times) for worker_times in detailed_task_allocation.values()
    ]
    max_worker_time = max(worker_time_summary)

    stage_times = []
    for task_idx, worker_list in enumerate(individual):
        task_time = tasks_times[task_idx]
        n_workers = len(worker_list)
        theoretical_time_per_worker = task_time / n_workers
        worker_times = [theoretical_time_per_worker / proficiency_levels[worker] for worker in worker_list]

        if n_workers == 1:
            total_time_for_task = worker_times[0]
        elif n_workers == 2:
            total_time_for_task = min(worker_times) * 2
        elif n_workers == 3:
            total_time_for_task = min(worker_times) * 3

        stage_times.append(total_time_for_task)

    total_1 = sum(stage_times[0:4])
    total_2 = sum(stage_times[4:27])
    total_3 = sum(stage_times[27:50])
    total_4 = sum(stage_times[50:53])
    total_5 = sum(stage_times[53:57])
    total_6 = sum(stage_times[57:85])

    re_stage_time = total_1 + max(max(total_2, total_3) + total_4, total_5) + total_6

    total_product = (28800 - re_stage_time) / max_worker_time + 1
    fitness = -total_product
    return fitness, total_product

def calculate_total_salary_with_constraints(individual, proficiency_levels, df):
    worker_salaries = {worker: 0 for worker in proficiency_levels.keys()}
    for task_idx, worker_list in enumerate(individual):
        task_price = df['ĐƠN GIÁ'][task_idx]
        num_workers = len(worker_list)
        if num_workers == 0:
            continue
        price_per_worker = task_price / num_workers
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_salaries[worker] += price_per_worker * proficiency
    total_salary = sum(worker_salaries.values())
    return total_salary, total_salary

def objective_function(individual, f_0, g_0, alpha):
    """

    Parameters:
    - f_x: Số lượng sản phẩm trong giải pháp hiện tại
    - g_x: Chi phí lương trong giải pháp hiện tại
    - f_0: Số lượng sản phẩm ban đầu (tham chiếu)
    - g_0: Chi phí lương ban đầu (tham chiếu)
    - alpha: Trọng số ưu tiên (0 <= alpha <= 1)

    Returns:
    - Giá trị của hàm mục tiêu H(x) cho giải pháp hiện tại
    """
    _, f_x = calculate_fitness_with_product(individual, tasks_times, proficiency_levels, df)
    _, g_x = calculate_total_salary_with_constraints(individual, proficiency_levels, df)
    normalized_product = f_x / f_0
    normalized_cost = g_0 / g_x
    H_x = (1 - alpha) * normalized_cost + lamda*alpha * normalized_product
    return -H_x, H_x

# Kiểm tra việc gán máy hợp lệ cho các công nhân hay ko
def check_worker_machines(individual, machines):
    worker_machines = {worker: set() for worker in workers}
    for task_idx, worker_list in enumerate(individual):
        machine_type = machines[task_idx]
        for worker in worker_list:
            worker_machines[worker].add(machine_type)
    for i in list(worker_machines):
      if str(worker_machines[i]).replace("0,","").count(",") >= 2:
        return False
    return True

def is_valid_machine_assignment(worker, machine_type, worker_machine_assignment):
    """
    Kiểm tra xem công nhân có thể sử dụng thêm thiết bị không.
    """
    assigned_machines = worker_machine_assignment[worker]
    non_zero_machines = {m for m in assigned_machines if m != 0}

    if machine_type == 0:
        return len(non_zero_machines) <= 2
    else:
        return len(non_zero_machines) < 2 or machine_type in assigned_machines

# Tính toán in ra các loại máy công nhân sử dụng
def calculate_worker_machines(individual, machines):
    worker_machines = {worker: set() for worker in workers}
    for task_idx, worker_list in enumerate(individual):
        machine_type = machines[task_idx]
        for worker in worker_list:
            worker_machines[worker].add(machine_type)

    # Convert to a DataFrame for easier viewing
    worker_machines_df = pd.DataFrame([
        {"Worker": worker, "Machines Used": sorted(list(machines_used))}
        for worker, machines_used in worker_machines.items()
    ]).sort_values(by="Worker").reset_index(drop=True)
    return worker_machines_df

# Phân công công nhân theo các ràng buộc
def greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df):
    sorted_workers = sorted(proficiency_levels.keys(), key=lambda x: proficiency_levels[x])
    individual = []
    worker_machine_assignment = {worker: set() for worker in sorted_workers}

    for task_idx, task_time in enumerate(tasks_times):
        machine_type = machines[task_idx]
        available_workers = [
            worker for worker in sorted_workers
            if is_valid_machine_assignment(worker, machine_type, worker_machine_assignment)
        ]

        if available_workers:
            n_selected_workers = random.randint(1, min(3, len(available_workers)))
            selected_workers = random.sample(available_workers, n_selected_workers)
            for worker in selected_workers:
                worker_machine_assignment[worker].add(machine_type)
            individual.append(selected_workers)
        else:
            # Nếu không còn công nhân hợp lệ, gán bất kỳ công nhân nào (xảy ra khi bị thiếu ràng buộc)
            fallback_worker = random.choice(sorted_workers)
            individual.append([fallback_worker])
            worker_machine_assignment[fallback_worker].add(machine_type)

    return individual

# Chạy Simulated Annealing với các ràng buộc
def simulated_annealing_with_constraints(
    initial_solution, tasks_times, proficiency_levels, df, initial_temp=1000, cooling_rate=0.95, max_iter=100000
):
    worker_machine_assignment = {worker: set() for worker in workers}

    current_solution = initial_solution
    current_fitness, current_product = objective_function(current_solution, f_0, g_0, alpha)
    best_solution = current_solution
    best_fitness = current_fitness
    best_product = current_product
    temperature = initial_temp

    for iteration in range(max_iter):
        new_solution = current_solution[:]
        task_idx = random.randint(0, len(tasks_times) - 1)
        machine_type = machines[task_idx]

        # Lựa chọn công nhân phù hợp
        possible_workers = [
            worker for worker in workers
            if is_valid_machine_assignment(worker, machine_type, worker_machine_assignment)
        ]
        n_selected_workers = random.randint(1, min(3, len(possible_workers))) if possible_workers else 1
        new_worker_list = random.sample(possible_workers, n_selected_workers) if possible_workers else [random.choice(workers)]

        # Gán giải pháp mới
        new_solution[task_idx] = new_worker_list
        for worker in new_worker_list:
            worker_machine_assignment[worker].add(machine_type)
        # Kiểm tra có thõa mãn phụ thuộc hay không
        if check_worker_machines(new_solution, machines) == False:
            continue
        # Tính fitness cho giải pháp mới
        new_fitness, new_product = objective_function(new_solution, f_0, g_0, alpha)
        delta_fitness = new_fitness - current_fitness

        # Chấp nhận giải pháp mới
        if delta_fitness < 0 or np.exp(-delta_fitness / max(temperature, 1e-6)) > random.random():
            current_solution = new_solution
            current_fitness = new_fitness
            current_product = new_product

        # Cập nhật giải pháp tốt nhất
        if current_fitness < best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness
            best_product = current_product
            print("-----------")
            print("sản lượng: "+str(calculate_fitness_with_product(best_solution, tasks_times, proficiency_levels, df)[1])+" / Lương: "+str(calculate_total_salary_with_constraints(best_solution, proficiency_levels, df)[0]))
            print("H= "+ str(best_product) + " / Alpha: " + str(alpha))
        # Giảm nhiệt độ
        temperature *= cooling_rate

    return best_solution, -best_fitness, best_product

# Run greedy assignment with constraints
initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)
while check_worker_machines(initial_solution, machines)==False:
  initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)
f_0 = 210.7096179
g_0 = 15255.43038

alphas = {"alpha="+str(i/10): [] for i in range(11)}
data = {}
for ii in range(10):
  for i in range(11):
    f_max = 291.7449101
    g_min = 12980.52
    f_ = f_max/f_0
    g_ = g_0/g_min
    print(f_)
    print(g_)
    alpha = i/10
    lamda = g_/f_
    # Optimize using Simulated Annealing with constraints
    best_solution, best_fitness, best_product = simulated_annealing_with_constraints(
        initial_solution, tasks_times, proficiency_levels, df
    )
    alphas["alpha="+str(i/10)].append({
        "sản lượng": calculate_fitness_with_product(best_solution, tasks_times, proficiency_levels, df)[1],
        "lương": calculate_total_salary_with_constraints(best_solution, proficiency_levels, df)[0],
        "H": best_product,
        "alpha": i/10,
        "best_solution": best_solution
    })

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
sản lượng: 241.9035217532863 / Lương: 14800.253333333334
H= 1.0082433237021236 / Alpha: 0.4
-----------
sản lượng: 241.9035217532863 / Lương: 14799.506666666664
H= 1.0082745259649275 / Alpha: 0.4
-----------
sản lượng: 241.9035217532863 / Lương: 14799.506666666663
H= 1.0082745259649277 / Alpha: 0.4
-----------
sản lượng: 241.88874166862928 / Lương: 14791.66666666667
H= 1.0085785240670755 / Alpha: 0.4
-----------
sản lượng: 241.88874166862928 / Lương: 14784.946666666667
H= 1.0088597841588518 / Alpha: 0.4
-----------
sản lượng: 241.88874166862928 / Lương: 14770.600000000002
H= 1.0094611085540124 / Alpha: 0.4
-----------
sản lượng: 242.24298981814195 / Lương: 14776.040000000003
H= 1.0098037763289687 / Alpha: 0.4
-----------
sản lượng: 244.029974927906 / Lương: 14777.106666666667
H= 1.012638514054517 / Alpha: 0.4
-----------
sản lượng: 244.96470560070557 / Lương: 14786.226666666666
H= 1.0137626363771157 / Alpha: 0.4
-----------
sản l

In [ ]:
alphas

{'alpha=0.0': [{'sản lượng': 30.26574415057843,
   'lương': 13175.879999999992,
   'H': 1.1578300940810033,
   'alpha': 0.0,
   'best_solution': [['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Bùi Thị Hồng Vân'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Hoàng Thị Mỹ Yến'],
    ['Trần Thị Cúc', 'Ngô Khắc Hiển'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển', 'Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Nguyễn Thị Ngọc Lan'],
    ['Trần Thị Cúc'],
    ['Đinh Thị Mi'],
    ['Ngô Khắc Hiển'],
    ['Bùi Thị Hồng Vân'],
    ['Ngô Khắc Hiển'],
    ['Ngô Khắc Hiển'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc', 'Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Hoàng Thị Mỹ Yến', 'Nguyễn Thị Mỹ Nhung', 'Nguy

In [ ]:
calculate_worker_machines(, machines)

,Worker,Machines Used
0,Bùi Thị Hồng Vân,"[0, 2, 5]"
1,Hoàng Thị Mỹ Yến,"[0, 1, 2]"
2,Hoàng Thị Nở,"[1, 2]"
3,Hồ Thị Diệp,"[0, 1]"
4,Lê Thị Như Quỳnh,"[0, 3, 7]"
5,Lê Thị Phương Thảo,[]
6,Lê Thị Thanh Hà,"[1, 4]"
7,Nguyễn Thị Gấm,[1]
8,Nguyễn Thị Hài,"[0, 4]"
9,Nguyễn Thị Hồng,[7]


In [ ]:
calculate_fitness_with_product(best_solution, tasks_times, proficiency_levels, df)[1]

268.3224986886785

In [ ]:
calculate_total_salary_with_constraints(best_solution, proficiency_levels, df)[0]

14926.039999999995

In [ ]:
initial_solution

[['Thái Thị Nga', 'Trần Thị Oanh', 'Nguyễn Thị Mỹ Nhung'],
 ['Nguyễn Thị Ngọc Lan'],
 ['Trần Thị Tuyền', 'Trần Thị Cúc', 'Nguyễn Thị Mỹ Nhung'],
 ['Lê Thị Phương Thảo', 'Trần Thị Cẩm Rin'],
 ['Trần Thị Oanh'],
 ['Trần Thị Cúc', 'Hoàng Thị Nở', 'Nguyễn Thị Gấm'],
 ['Hồ Thị Diệp', 'Nguyễn Thị Ngọc Lan'],
 ['Nguyễn Thị Hài', 'Nguyễn Thị Gấm'],
 ['Đinh Thị Mi'],
 ['Nguyễn Thị Thúy', 'Hoàng Thị Nở'],
 ['Nguyễn Thị Ngọc Lan', 'Nguyễn Thị Hài', 'Trần Thị Cẩm Rin'],
 ['Trần Thị Cẩm Rin'],
 ['Phan Thị Thu Hoài', 'Trần Thị Oanh', 'Nguyễn Thị Thúy'],
 ['Lê Thị Phương Thảo', 'Trần Thị Cẩm Rin'],
 ['Nguyễn Thị Gấm', 'Nguyễn Thị Mỹ Nhung', 'Trần Thị Tuyền'],
 ['Trần Thị Oanh'],
 ['Trần Thị Cúc', 'Hồ Thị Diệp', 'Nguyễn Thị Gấm'],
 ['Nguyễn Thị Mỹ Nhung', 'Nguyễn Thị Hồng'],
 ['Lê Thị Phương Thảo', 'Ngô Khắc Hiển'],
 ['Bùi Thị Hồng Vân', 'Nguyễn Thị Gấm', 'Hoàng Thị Mỹ Yến'],
 ['Trần Thị Cúc', 'Hoàng Thị Mỹ Yến'],
 ['Nguyễn Thị Thu', 'Thái Thị Nga', 'Trần Thị Oanh'],
 ['Hoàng Thị Nở'],
 ['Trần Thị Thư

In [ ]:
for alpha, values in alphas.items():
    san_luong = [entry['sản lượng'] for entry in values][:5]
    print(f"{alpha}: {san_luong}")

alpha=0.0: [43.221238399336755, 27.6252625160897, 36.06865365766729, 34.38113640224223, 34.89653018587521]
alpha=0.1: [148.43165817047688, 187.13989028213166, 154.13654398245217, 209.25068013527027, 163.74676619757605]
alpha=0.2: [165.01388601036268, 239.67493461620853, 242.0661432535197, 238.20531572769707, 230.57661194791626]
alpha=0.3: [212.65127838343554, 210.2628005802798, 188.7230534998043, 233.52931954848822, 232.3244336999451]
alpha=0.4: [255.996681663384, 209.84702674173116, 148.28667100927564, 254.44488887871037, 232.04241958996107]
alpha=0.5: [294.95081149225985, 242.3768421288705, 254.89423610358807, 271.7767779366072, 230.99583494769468]
alpha=0.6: [247.5869627895036, 154.36601825258302, 255.5037232026807, 291.84166992192627, 247.38500849404855]
alpha=0.7: [223.55417642710017, 305.3629670422774, 279.595671365145, 183.02154085119602, 279.92898772390606]
alpha=0.8: [271.82307792755046, 307.14279483043526, 304.91773978556944, 247.63600855905756, 228.3322342294555]
alpha=0.9: 

In [ ]:
# Reformat data for easier selection based on 'sản lượng'
selected_san_luong = []

# Iterate through the keys in sorted order
for alpha, values in sorted(alphas.items(), key=lambda x: x[0]):
    # Find the minimum sản lượng that is greater than the last selected sản lượng
    if selected_san_luong:
        last_value = selected_san_luong[-1]
        valid_values = [entry['sản lượng'] for entry in values if entry['sản lượng'] > last_value]
    else:
        valid_values = [entry['sản lượng'] for entry in values]

    # If a valid value exists, select the smallest one
    if valid_values:
        selected_san_luong.append(min(valid_values))

selected_san_luong


[26.123485986385862,
 118.49401854291867,
 153.8588462777957,
 156.1675063796375,
 197.5286704023302,
 219.4525893416928,
 224.71923017626804,
 233.40082681227216,
 237.72078258285154,
 280.2453812839209,
 307.7922809172809]

In [ ]:
# Extract corresponding 'lương' and 'H' for the selected sản lượng values
corresponding_data = []

for alpha, values in sorted(alphas.items(), key=lambda x: x[0]):
    for entry in values:
        if entry['sản lượng'] in selected_san_luong:
            corresponding_data.append((entry['sản lượng'], entry['lương'], entry['H']))
            break

corresponding_data


[(26.123485986385862, 12982.119999999995, 1.175110874032901),
 (118.49401854291867, 14273.053333333326, 1.0072276243477625),
 (153.8588462777957, 14358.759999999995, 0.9675536627891291),
 (156.1675063796375, 14366.06666666666, 0.9223752189889212),
 (197.5286704023302, 14464.546666666663, 0.9347519740486245),
 (219.4525893416928, 14528.89333333333, 0.9443266752139301),
 (224.71923017626804, 14535.37333333333, 0.9350795069166716),
 (233.40082681227216, 14664.52, 0.9364540965928136),
 (237.72078258285154, 14761.559999999996, 0.9334590115437003),
 (280.2453812839209, 15470.973333333332, 1.0624789974691249),
 (307.7922809172809, 15449.773333333334, 1.176240714889341)]

In [ ]:
pd.DataFrame(corresponding_data, columns=['Sản lượng', 'Lương', 'H'])

,Sản lượng,Lương,H
0,26.123486,12982.120000,1.175111
1,118.494019,14273.053333,1.007228
2,153.858846,14358.760000,0.967554
3,156.167506,14366.066667,0.922375
4,197.528670,14464.546667,0.934752
5,219.452589,14528.893333,0.944327
6,224.719230,14535.373333,0.935080
7,233.400827,14664.520000,0.936454
8,237.720783,14761.560000,0.933459
9,280.245381,15470.973333,1.062479


In [ ]:
f_max

298.014716934321

In [ ]:
alphas

{'alpha=0.0': [{'sản lượng': 30.26574415057843,
   'lương': 13175.879999999992,
   'H': 1.1578300940810033,
   'alpha': 0.0,
   'best_solution': [['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Bùi Thị Hồng Vân'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Hoàng Thị Mỹ Yến'],
    ['Trần Thị Cúc', 'Ngô Khắc Hiển'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển', 'Trần Thị Cúc'],
    ['Trần Thị Cúc'],
    ['Ngô Khắc Hiển'],
    ['Nguyễn Thị Ngọc Lan'],
    ['Trần Thị Cúc'],
    ['Đinh Thị Mi'],
    ['Ngô Khắc Hiển'],
    ['Bùi Thị Hồng Vân'],
    ['Ngô Khắc Hiển'],
    ['Ngô Khắc Hiển'],
    ['Ngô Khắc Hiển'],
    ['Trần Thị Cúc', 'Ngô Khắc Hiển'],
    ['Trần Thị Cúc'],
    ['Hoàng Thị Mỹ Yến', 'Nguyễn Thị Mỹ Nhung', 'Nguy

In [ ]:
# Function to extract salaries and sản lượng that are strictly increasing across alphas
def find_increasing_values(data):
    alpha_keys = sorted(data.keys(), key=lambda x: float(x.split('=')[1]))
    selected_values = []

    for i in range(len(alpha_keys)):
        alpha = alpha_keys[i]
        entries = data[alpha]

        # Extract the sản lượng, lương, and H triples
        for entry in entries:
            san_luong, luong = entry['sản lượng'], entry['lương']

            # If this is the first alpha, add the first triple
            if not selected_values:
                selected_values.append((san_luong, luong))
            else:
                # Check if both sản lượng and lương are increasing
                last_san_luong, last_luong= selected_values[-1]
                if san_luong > last_san_luong and luong > last_luong:
                    selected_values.append((san_luong, luong))
            # Stop once we have one valid triple per alpha
            if len(selected_values) > i:
                break

    return selected_values

# Get the increasing values
increasing_values = find_increasing_values(filtered_results)
increasing_values


In [ ]:
aa = filtered_results["alpha=0.0"] + filtered_results["alpha=0.1"] + filtered_results["alpha=0.2"] + filtered_results["alpha=0.3"] + filtered_results["alpha=0.4"] + filtered_results["alpha=0.5"] + filtered_results["alpha=0.6"] + filtered_results["alpha=0.7"] + filtered_results["alpha=0.8"] + filtered_results["alpha=0.9"]+ filtered_results["alpha=1.0"]

In [ ]:
sorted(aa, key=lambda x: x['lương'])

In [ ]:
# Filter values where salary ('lương') and production ('sản lượng') are both increasing with alpha and salary < 291

filtered_results = {}
max_salary_limit = 291  # As per the constraint

for alpha, values in alphas.items():
    increasing_values = []
    for entry in values:
        if entry['sản lượng'] < max_salary_limit:
            increasing_values.append({'lương': entry['lương'], 'sản lượng': entry['sản lượng']})
    if increasing_values:
        filtered_results[alpha] = increasing_values

filtered_results
